In [1]:
from tqdm.auto import tqdm
import torch, torchtext
from torch import nn
import torch.nn.functional as F
import random, math, time
from datasets import load_dataset
import pandas as pd
import numpy as np
import re

# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device("cpu")
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cpu


In [2]:
import torch, torchtext, transformers
import numpy
print(torch.__version__, torchtext.__version__, transformers.__version__)
print(numpy.__version__)


2.3.0+cu121 0.18.0+cpu 4.49.0
1.26.4


In [3]:
# https://github.com/frankaging/Quasi-Attention-ABSA/tree/main

In [4]:
from datasets import load_from_disk
# .select(range(10000))
train_data = load_from_disk("tokenized_data_edit/train")
val_data = load_from_disk("tokenized_data_edit/validation")
test_data = load_from_disk("tokenized_data_edit/test")


In [5]:
train_data = train_data.rename_column("label", "labels")
val_data = val_data.rename_column("label", "labels")
test_data = test_data.rename_column("label", "labels")

In [6]:
train_data

Dataset({
    features: ['text1', 'entity', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 452057
})

In [7]:
dataset = load_from_disk("dataset_edit/dataset_edit")

In [8]:
 # dataset['test'] = dataset['test'].select(range(1000))

 # test_data = test_data.select(range(1000))

In [9]:
# Extract unique labels (MITRE techniques) from both train and validation datasets
labels = list(set(dataset['train']['label']).union(set(dataset['validation']['label'])))  # Extract unique labels
label_map = {label: i for i, label in enumerate(labels)}

# Function to encode labels into integers
# def encode_labels(examples):
#     # Safely map the labels, providing a default value if a label is not found in the label_map
#     examples['labels'] = [label_map.get(label, -1) for label in examples['labels']]
    
#     return examples

# # Apply label encoding
# train_data = train_data.map(encode_labels, batched=True)
# val_data = val_data.map(encode_labels, batched=True)

# # Print an example to verify
print(train_data[3])  # It should show tokenized input along with the encoded label


{'text1': 'These Microsoft Office templates are hosted on a command and control server and the downloaded link is embedded in the first stage malicious document', 'entity': 'I-TOOL', 'labels': 'T1584.004', 'input_ids': [2, 455, 755, 1899, 8818, 226, 4700, 158, 43, 979, 137, 656, 629, 137, 114, 3633, 695, 146, 3718, 120, 114, 773, 3872, 815, 1355, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [10]:
# Make sure each 'entity' is treated as a full string, not split into characters
unique_aspects = list(set(aspects for aspects in dataset['train']['entity']))
context_map = {asp: i+1 for i, asp in enumerate(unique_aspects)}  # +1 to reserve 0 for padding

# # Update encoding accordingly
# def encode_context(examples):
#     examples["context_ids"] = [
#         [context_map[a] for a in aspects if a in context_map] if isinstance(aspects, list) else [context_map.get(aspects, 0)]
#         for aspects in examples["entity"]
#     ]
#     return examples

# # Apply context encoding
# # train_data = train_data.map(encode_context, batched=True)
# # val_data = val_data.map(encode_context, batched=True)
# test_data = test_data.map(encode_context, batched=True)

# Print an example to verify
print(test_data[3])  # It should show tokenized input along with the encoded label

{'text1': 'The spear phishing emails contained three attachments in total, each of which exploited an older vulnerability in Microsoft Office (CVE-2012-0158', 'entity': 'B-ACT', 'labels': 'T1203', 'input_ids': [2, 114, 5525, 1587, 2596, 3898, 1775, 4905, 120, 2683, 16, 1041, 135, 388, 2543, 124, 4767, 525, 120, 755, 1899, 12, 539, 17, 2092, 17, 28500, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [11]:
import torch
from torch.utils.data import DataLoader, Dataset

class TTPDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, label_key='labels', max_context_id=None):
        self.dataset = hf_dataset
        self.label_key = label_key
        self.max_context_id = max_context_id

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        context_ids = torch.tensor(item["context_ids"], dtype=torch.long)

        if self.max_context_id is not None:
            context_ids = torch.clamp(context_ids, min=0, max=self.max_context_id)

        label = item[self.label_key]
        if label == -1:
            # Replace -1 with a default class, like 0
            label = 0  # or skip the example entirely


        return {
            "input_ids": torch.tensor(item["input_ids"]),
            "token_type_ids": torch.tensor(item["token_type_ids"]),
            "attention_mask": torch.tensor(item["attention_mask"]),
            "labels": torch.tensor(item[self.label_key]),
            "context_ids": context_ids
        }

train_dataset = TTPDataset(train_data)
val_dataset = TTPDataset(val_data)
test_dataset = TTPDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [12]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds, label = p
    # If using softmax, we need to use argmax to get the final class prediction
    preds = preds.argmax(axis=-1)
    
    # Calculate precision, recall, F1-score, and accuracy
    precision, recall, f1, _ = precision_recall_fscore_support(label, preds, average='weighted')
    accuracy = accuracy_score(label, preds)
    
    # Return the metrics as a dictionary
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [13]:
model = torch.load('models/full_model_edit.pth')
model.to(device)


QACGBertForSequenceClassification(
  (bert): ContextBertModel(
    (embeddings): BERTEmbeddings(
      (word_embeddings): Embedding(50000, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BERTLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ContextBERTEncoder(
      (context_layer): ModuleList(
        (0-11): 12 x Linear(in_features=1536, out_features=768, bias=True)
      )
      (layer): ModuleList(
        (0-11): 12 x ContextBERTLayer(
          (attention): ContextBERTAttention(
            (self): ContextBERTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (context_for_q): Linear(in_features=768, out_features=768, bias=T

In [14]:
import pandas as pd
df = pd.read_csv('real_news_maps.csv')

In [15]:
df

,news,maps
0,Cybersecurity researchers have detailed the ac...,"['T1190','T1078','T1059','T1133','T1078','T102..."
1,North Korea-linked threat actors behind the Co...,"['T1589','T1591','T1598','T1585','T1586','T158..."
2,Threat actors are likely exploiting a new vuln...,"['T1190','T1059.004','T1505.003','T1548','T107..."
3,"When we talk about identity in cybersecurity, ...","['T1552.001','T1552.002','T1083','T1119','T109..."
4,At least six organizations in South Korea have...,"['T1189','T1583.001','T1190','T1059.001','T154..."
5,Cybersecurity researchers have demonstrated a ...,"['T1211','T1059.004','T1071.001','T1003','T1021']"
6,The threat actors behind the Darcula phishing-...,"['T1566.001','T1078','T1119','T1071.001','T102..."
7,Multiple threat activity clusters with ties to...,"['T1566.002','T1189','T1078.004','T1078','T154..."
8,The Iran-nexus threat actor known as UNC2428 h...,"['T1566.001','T1059.003','T1547.001','T1548.00..."
9,Cybersecurity researchers have revealed that R...,"['T1475','T1543.003','T1547','T1068','T1036.00..."


In [16]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
import spacy

# Load tokenizer and spaCy model
tokenizer = BertTokenizer.from_pretrained("ibm-research/CTI-BERT")
nlp = spacy.load("en_core_web_sm")

# Inverse label map
label_map_rev = {v: k for k, v in label_map.items()}

def predict_single(text, model, tokenizer, context_map, label_map_rev, device, max_len=64):
    # 1. Tokenize input
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=max_len, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)

    seq_len = input_ids.size(1)

    # 2. Extract entities using spaCy for context
    doc = nlp(text)
    aspects = list(set([ent.text for ent in doc.ents if ent.text in context_map]))


    # 3. Encode context
    context_ids = [context_map.get(a, 0) for a in aspects]
    if not context_ids:
        context_ids = [0]  # default to padding if no valid entity
    context_ids = torch.tensor([context_ids], dtype=torch.long).to(device)

    # 4. Forward pass
    model.eval()
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            seq_lens=[seq_len],
            context_ids=context_ids,
        )

    logits = outputs
    probs = F.softmax(logits, dim=1).squeeze()
    pred_idx = torch.argmax(probs).item()
    pred_label = label_map_rev[pred_idx]

    return pred_label, probs.cpu().numpy()


In [17]:
# import numpy as np

# input_text = input("Enter text to classify: ")
# predicted_label, confidence_scores = predict_single(input_text, model, tokenizer, context_map, label_map_rev, device)

# print(f"\nPredicted Label: {predicted_label}")
# print("\nTop 5 Confidence Scores:")

# # Get indices of top 5 scores
# top_indices = np.argsort(confidence_scores)[-5:][::-1]

# for idx in top_indices:
#     label = label_map_rev[idx]
#     score = confidence_scores[idx]
#     print(f"{label}: {score:.4f}")


In [18]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
import spacy
from sklearn.metrics import ndcg_score

# Load tokenizer and spaCy model
tokenizer = BertTokenizer.from_pretrained("ibm-research/CTI-BERT")
nlp = spacy.load("en_core_web_sm")

# Inverse label map
label_map_rev = {v: k for k, v in label_map.items()}
label_map = {label: idx for idx, label in label_map_rev.items()}
  # forward mapping, important for later

def predict_single(text, model, tokenizer, context_map, device, max_len=64):
    # 1. Tokenize input
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=max_len, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    token_type_ids = inputs["token_type_ids"].to(device)

    seq_len = input_ids.size(1)

    # 2. Extract entities using spaCy for context
    doc = nlp(text)
    aspects = list(set([ent.text for ent in doc.ents if ent.text in context_map]))

    # 3. Encode context
    context_ids = [context_map.get(a, 0) for a in aspects]
    if not context_ids:
        context_ids = [0]  # default to padding if no valid entity
    context_ids = torch.tensor([context_ids], dtype=torch.long).to(device)

    # 4. Forward pass
    model.eval()
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            seq_lens=[seq_len],
            context_ids=context_ids,
        )

    logits = outputs
    probs = F.softmax(logits, dim=1).squeeze()

    return probs.cpu().numpy()  # Now return **only** probabilities


In [28]:
def compute_ndcg_for_sample(probs, true_labels, label_map, k=5):
    """
    probs: np.array of shape (num_classes,)
    true_labels: list of correct labels, ex: ['T1190', 'T1078', ...]
    label_map: dict mapping label text to indices
    k: cutoff for NDCG@k
    """
    # Step 1: Build true relevance vector
    num_classes = probs.shape[0]
    true_relevance = [0] * num_classes
    for label in true_labels:
        idx = label_map.get(label)
        if idx is not None:
            true_relevance[idx] = 1  # relevant

    true_relevance = [true_relevance]  # sklearn expects 2D array
    probs = [probs.tolist()]           # sklearn expects 2D array

    return ndcg_score(true_relevance, probs, k=k)


In [25]:
# def compute_ndcg_for_sample(probs, true_labels, label_map,relevant_list, k=5):
#     """
#     probs: np.array of shape (num_classes,)
#     true_labels: list of correct labels, ex: ['T1190', 'T1078', ...]
#     label_map: dict mapping label text to indices
#     k: cutoff for NDCG@k
#     """
#     # Step 1: Build true relevance vector
#     # num_classes = probs.shape[0]
#     # true_relevance = [1] * 5
#     true_relevance = []
#     score = 0
#     for x in true_labels[-5:]:
#         if x in relevant_list:
#             score = score + 1
    
#     # for label in true_labels[-5:]:
#     #     # print(label)
#     #     idx = label_map.get(label)
#     #     # print(idx)
#     #     if idx is not None:
#     #         true_relevance.append(1)  # relevant

#     # # true_relevance = [true_relevance]  # sklearn expects 2D array
#     # # probs = [probs.tolist()]           # sklearn expects 2D array

#     # # print('tru value',true_relevance)
#     print('predict value',relevant_list,'+',score)
    
#     return (score/5),score


In [26]:
# import pandas as pd

# # Example: load your dataset
# data = pd.read_csv("real_news_maps.csv")  # with 'news' and 'maps' columns

# all_ndcg_scores = []

# for idx, row in data.iterrows():
#     text = row['news']
#     true_labels = eval(row['maps'])  # careful: safely convert string to list
    
#     probs = predict_single(text, model, tokenizer, context_map, device)
#     top_indices = np.argsort(probs)[-20:][::-1]
#     relevant_list = []
#     for idx2 in top_indices:
#         label = label_map_rev[idx2]
#         relevant_list.append(label)
#     ndcg,score = compute_ndcg_for_sample(probs, true_labels, label_map,relevant_list)

#     print('true value',true_labels[:6])
#     # print(score)
    
#     all_ndcg_scores.append(ndcg)
#     print(f"Sample {idx}: NDCG@5 = {ndcg:.4f}")

# # After all samples
# mean_ndcg = sum(all_ndcg_scores) / len(all_ndcg_scores)
# print(f"\nMean NDCG@5 over dataset = {mean_ndcg:.4f}")


predict value ['T1048', 'T1562', 'T1583.006', 'T1569.002', 'T1037.004', 'T1078.002', 'T1087.002', 'T1037.002', 'T1593', 'T1222.002', 'T1092', 'T1008', 'T1590.001', 'T1072', 'T1498', 'T1098.001', 'T1037.001', 'T1070.006', 'T1070.001', 'T1027.005'] + 0
true value ['T1190', 'T1078', 'T1059', 'T1133', 'T1078', 'T1027']
Sample 0: NDCG@5 = 0.0000
predict value ['T1621', 'T1136.001', 'T1218.010', 'T1218.005', 'T1037.001', 'T1564.006', 'T1543', 'T1070.003', 'T1005', 'T1606.002', 'T1550.002', 'T1560.001', 'T1104', 'T1583.006', 'T1590', 'T1555.005', 'T1071', 'T1092', 'T1585', 'T1134.003'] + 0
true value ['T1589', 'T1591', 'T1598', 'T1585', 'T1586', 'T1587']
Sample 1: NDCG@5 = 0.0000
predict value ['T1222.002', 'T1569.002', 'T1087.002', 'T1497.001', 'T1072', 'T1092', 'T1590.001', 'T1078.002', 'T1037.001', 'T1048', 'T1037.004', 'T1543', 'T1498', 'T1574.010', 'T1056', 'T1205.001', 'T1608', 'T1037.002', 'T1005', 'T1546.016'] + 1
true value ['T1190', 'T1059.004', 'T1505.003', 'T1548', 'T1070', 'T1003

In [22]:
    print(label_map_rev.get('T1593.001'))

None


In [29]:
import pandas as pd

# Example: load your dataset
data = pd.read_csv("real_news_maps.csv")  # with 'news' and 'maps' columns

all_ndcg_scores = []

for idx, row in data.iterrows():
    text = row['news']
    true_labels = eval(row['maps'])  # careful: safely convert string to list
    
    probs = predict_single(text, model, tokenizer, context_map, device)
    ndcg = compute_ndcg_for_sample(probs, true_labels, label_map,k=20)
    
    all_ndcg_scores.append(ndcg)
    print(f"Sample {idx}: NDCG@5 = {ndcg:.4f}")

# After all samples
mean_ndcg = sum(all_ndcg_scores) / len(all_ndcg_scores)
print(f"\nMean NDCG@5 over dataset = {mean_ndcg:.4f}")


Sample 0: NDCG@5 = 0.0000
Sample 1: NDCG@5 = 0.0544
Sample 2: NDCG@5 = 0.0544
Sample 3: NDCG@5 = 0.0000
Sample 4: NDCG@5 = 0.0000
Sample 5: NDCG@5 = 0.0000
Sample 6: NDCG@5 = 0.0000
Sample 7: NDCG@5 = 0.0000
Sample 8: NDCG@5 = 0.0000
Sample 9: NDCG@5 = 0.0538

Mean NDCG@5 over dataset = 0.0163
